<a href="https://colab.research.google.com/github/Davvid01/PUM/blob/main/Copy_of_BigData_students.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Big Data

### Prowadzący, modyfikacje i aktualizacja: dr Marcin Szmydt

Opracowanie pierwotne: mgr inż. Dominik Filipiak
Notebook do zdalnej pracy ze Sparkiem.

### Przygotowanie sparka do pracy z colabem

Przed przystąpieniem do pracy musimy uruchomić poniższe komórki.
W razie kłopotów sprawdźcie czy nie zmieniła się wersja tutaj: http://www.apache.org/dist/spark/. W razie czego trzeba pozmieniać numerki wszędzie (np. 3.4.1 na 3.5.1).

Tu ściągamy Javę oraz Sparka razem z Hadoopem, a także instalujemy pakiet findspark w pythonie. Pewnie już się domyślacie, że przez wykrzyknik można wywołać linuksowe polecenia w Colabie.



In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar xf spark-3.5.1-bin-hadoop3.tgz
!pip install -q findspark==1.3.0

Instalujemy pyspark w Pythonie

In [ ]:
!pip install -q pyspark==3.5.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


Tu ustalamy zmienne środowiskowe.

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"

Jesteśmy gotowi do odpalenia Sparka - tworzymy sesję i kontekst.

In [ ]:
import findspark
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext

findspark.init("spark-3.5.1-bin-hadoop3")
sc = pyspark.SparkContext('local[*]')
spark = SparkSession.builder.appName('abc').getOrCreate()

In [ ]:
!free -mh # Komenda na sprawdzenie dostępnej pamięci RAM

               total        used        free      shared  buff/cache   available
Mem:            12Gi       1.0Gi       1.7Gi       1.0Mi         9Gi        11Gi
Swap:             0B          0B          0B


# Możemy przystąpić do pracy!

In [ ]:
sc.parallelize([1,2,3]).map(lambda x: x + 1).collect()

[2, 3, 4]

# Zajęcia 25.03.2024

In [ ]:
sc

###Przechodzenie między środowiskiem lokalnym (Python) a środowiskiem rozproszonym (Spark)

In [ ]:
lista = [1, 2, 3]

In [ ]:
type(lista)

In [ ]:
distributed_list = sc.parallelize(lista)

In [ ]:
type(distributed_list)

In [ ]:
distributed_list.take(10)

In [ ]:
listapython = distributed_list.map(lambda x: x + 1).collect()

In [ ]:
listapython

In [ ]:
type(listapython)

###RDD -Wczytywanie plików, podstawowe operacje (take, map, filter)

In [ ]:
!head -5 baby.csv

In [ ]:
baby_names = sc.textFile('baby.csv')

In [ ]:
baby_names.take(10)

In [ ]:
rows = baby_names.map(lambda x: x.split(','))

In [ ]:
rows.take(3)

In [ ]:
rows.filter(lambda x: 'Year' not in x).take(5)

###Zadanie 1 - Jak odfiltrować z RDD imiona które które mają mniej niż 250 wystąpień (count)

In [ ]:
#kod

###Cache (i wpływ na czas przetarzania)

In [ ]:
nums = sc.parallelize(range(1, 1000001))

In [ ]:
nums.count()

In [ ]:
def isPrime(n) :

    # Corner cases
    if (n <= 1) :
        return False
    if (n <= 3) :
        return True

    # This is checked so that we can skip
    # middle five numbers in below loop
    if (n % 2 == 0 or n % 3 == 0) :
        return False

    i = 5
    while(i * i <= n) :
        if (n % i == 0 or n % (i + 2) == 0) :
            return False
        i = i + 6

    return True

In [ ]:
isPrime(11)

In [ ]:
primes = nums.filter(isPrime)

In [ ]:
primes.count()

In [ ]:
import time

In [ ]:
start = time.time()
print(f'Jest {primes.count()} liczb pierwszych w milionie')
end = time.time()
print(f'Upłynęło {end-start} sekund')

In [ ]:
primes.cache()

In [ ]:
start = time.time()
print(f'Jest {primes.count()} liczb pierwszych w milionie')
end = time.time()
print(f'Upłynęło {end-start} sekund')

In [ ]:
start = time.time()
print(f'Jest {primes.count()} liczb pierwszych w milionie')
end = time.time()
print(f'Upłynęło {end-start} sekund')

In [ ]:
start = time.time()
print(f'Jest {primes.count()} liczb pierwszych w milionie')
end = time.time()
print(f'Upłynęło {end-start} sekund')

In [ ]:
primes.unpersist()

In [ ]:
start = time.time()
print(f'Jest {primes.count()} liczb pierwszych w milionie')
end = time.time()
print(f'Upłynęło {end-start} sekund')

###Sampling

In [ ]:
one_to_ten = range(1,11)
parallel = sc.parallelize(one_to_ten)

In [ ]:
parallel.take(100)

In [ ]:
parallel.sample(True, .2).take(5)

In [ ]:
parallel.takeSample(True, 2)

In [ ]:
parallel_big = sc.parallelize(range(1,100001))

In [ ]:
parallel_big.sample(True, .2).count()

In [ ]:
parallel_big.sample(True, .2, 5345).take(5)

###Intersection, Union, Joins

In [ ]:
one = sc.parallelize(range(1,10))
two = sc.parallelize(range(5,15))

In [ ]:
one.collect()

In [ ]:
two.collect()

In [ ]:
one.intersection(two).collect()

In [ ]:
one.union(two).collect()

In [ ]:
one.union(two).distinct().collect()

In [ ]:
names1 = sc.parallelize(('abe', 'abby', 'apple')).map(lambda x: (x, 1))
names2 = sc.parallelize(('apple', 'beatty', 'beatrice')).map(lambda x: (x, 5))
print(f'names1: {names1.collect()}')
print(f'names2: {names2.collect()}')

In [ ]:
names1.join(names2).collect()

In [ ]:
names1.leftOuterJoin(names2).collect()

In [ ]:
names1.rightOuterJoin(names2).collect()

In [ ]:
names1.cartesian(names2).collect()

###Zapis do jednego pliku (użycie coalesce)

In [ ]:
wynik = names1.cartesian(names2)

In [ ]:
type(wynik)

In [ ]:
wynik.saveAsTextFile('wynik.txt')

In [ ]:
!cat wynik.txt/part-00000

In [ ]:
wynik.coalesce(1).saveAsTextFile('wynik_one_partition.txt')

In [ ]:
!cat wynik_one_partition.txt/part-00000

###Zadanie 2
1. Stwórz zmienną w rozproszonym środowisku Sparka (RDD), która zawiera liczby od jeden do miliona
2. Usuń z tej zmiennej liczby, które mają resztę z dzielenia przez 5 mniejszą niż 2
3. Wylosuj 1% rekordów z uzyskanego RDD bez zwracania z ziarnem losowości 12345
4. Zapisz na lokalnym systemie plików do jednego pliku tekstowego
5. Odczytaj pierwsze 3 liczby z zapisanego pliku

In [ ]:
#kod

###groupByKey

In [ ]:
!head -5 baby.csv

In [ ]:
baby_names = sc.textFile('baby.csv')

In [ ]:
baby_names_no_header = baby_names.filter(lambda x: x!='Year,First Name,County,Sex,Count')

In [ ]:
baby_names_no_header.take(10)

In [ ]:
rows = baby_names_no_header.map(lambda x: x.split(','))

In [ ]:
rows.take(10)

In [ ]:
#W jakich hrabstwach występuje dane imię

In [ ]:
(rows
 .map(lambda x: (x[1], x[2]))
 .groupByKey()
 .map(lambda x: {x[0]: list(x[1])})
 ).collect()

###reduceByKey

In [ ]:
rows.take(20)

In [ ]:
#Ile razy dane imię wystąpiło we wszystkich hrabstwach

In [ ]:
?rows.reduceByKey()

In [ ]:
(rows
 .map(lambda x: (x[1],int(x[4])))
 .reduceByKey(lambda x,y: x+y)
).collect()

###Reduce + flatMap

In [ ]:
x1 = sc.parallelize(['kot','pies','zajac', 'niedzwiedz'])
x2 = sc.parallelize(['wilk','pies','dzik', 'bocian'])

In [ ]:
x1.reduce(lambda x,y: x + '_' + y)

In [ ]:
#łączna liczba znaków we wszystkich wyrazach w x1

In [ ]:
(x1
 .map(lambda x: [x, len(x)])
 .flatMap(lambda x: [x[1]])
).reduce(lambda x, y: x + y)

# Zajęcia 12.04.2023

###CountByKey

In [ ]:
x12 = x1.union(x2)
x12.take(10)

In [ ]:
(x12
 .map(lambda x: (x, 'wartosc-wymagane-pole'))
 ).collect()

In [ ]:
(x12
 .map(lambda x: (x, 'wartosc-wymagane-pole'))
 ).countByKey()

In [ ]:
slownik = (x12
 .map(lambda x: (x, 'wartosc-wymagane-pole'))
 ).countByKey()

In [ ]:
x12_dict_python = dict(slownik)
x12_dict_python

In [ ]:
x12_dict_python['pies']

###Broadcast

In [ ]:
my_local_dict = {'cityX': [1,2,3],
                 'cityY': [4,5,6]}

In [ ]:
my_rdd = sc.parallelize(['cityX','cityY'])
my_rdd.collect()

In [ ]:
my_rdd.map(lambda x: (x, my_local_dict[x][2])).collect() #Złe podejście

In [ ]:
my_broadcasted_dict = sc.broadcast(my_local_dict)

In [ ]:
my_broadcasted_dict.value

In [ ]:
my_rdd.map(lambda x: (x, my_broadcasted_dict.value[x][2])).collect() #Dobre podejście

###Accumulators

In [ ]:
accumulator = sc.accumulator(0)

In [ ]:
accumulator.value

In [ ]:
sc.parallelize([1,2,3,4]).foreach(lambda x: accumulator.add(x))

In [ ]:
accumulator.value

###Zadanie 3 - wordcount
1. Wczytaj plik TREASURE_ISLAND.txt z moodle do RDD
2. wykonaj wordcount (jakie słowo ile razy wystąpiło)

In [ ]:
#kod

##Spark SQL (DataFrames)

In [ ]:
!head -5 people.txt

In [ ]:
spark

In [ ]:
df = spark.read.csv('people.txt', header=True)

In [ ]:
type(df)

In [ ]:
df.show(100)

In [ ]:
df.storageLevel.useMemory

In [ ]:
df.cache()

In [ ]:
df.storageLevel.useMemory

In [ ]:
import pyspark.sql.functions as f

In [ ]:
x = df.select('Age', df['Age']+2, f.col('Age')+2).toPandas()

In [ ]:
x

In [ ]:
df.show(100)

In [ ]:
(df
 .filter(f.col('Age') > 20)
 .filter(f.col('Name').startswith('F'))
 ).show(100)

In [ ]:
df.groupBy('Name').count().show()

###TempView / GlobalTempView (ala SQL)

In [ ]:
df.createOrReplaceTempView('people')
sqlDF = spark.sql("SELECT * FROM people WHERE Age>20 AND SUBSTR(Name,1,1)='F'")
sqlDF.show()

In [ ]:
df.createOrReplaceGlobalTempView('people_g')
sqlDF = spark.sql("SELECT * FROM global_temp.people_g WHERE Age>20 AND SUBSTR(Name,1,1)='F'")
sqlDF.show()

### rdd $\mapsto$ DF

In [ ]:
lines = sc.textFile('people_noheader.txt')

In [ ]:
lines.take(10)

In [ ]:
parts = lines.map(lambda x: x.split(','))
parts.take(10)

In [ ]:
parts = parts.map(lambda x: [x[0], int(x[1].strip())])
parts.take(10)

In [ ]:
from pyspark.sql import Row

In [ ]:
people = parts.map(lambda x: Row(Name=x[0], Age=x[1]))
people.take(10)

In [ ]:
type(people)

In [ ]:
DFpeople = spark.createDataFrame(people)

In [ ]:
type(DFpeople)

###modyfikacja schema - sposob 1

In [ ]:
DFpeople.show()

In [ ]:
DFpeople.printSchema()

In [ ]:
from pyspark.sql.types import *

In [ ]:
fields = [StructField('Names', StringType()),
          StructField('Age', IntegerType())]

In [ ]:
schema = StructType(fields)

In [ ]:
schema

In [ ]:
DFpeople = spark.createDataFrame(people, schema)

In [ ]:
DFpeople.printSchema()

In [ ]:
DFpeople.show()

In [ ]:
DFpeople.createOrReplaceTempView('people')

In [ ]:
x = spark.sql("SELECT * FROM people WHERE AGE >=20 AND AGE <=40")

In [ ]:
x.show()

In [ ]:
DFpeople.printSchema()

###modyfikacja schema - sposob 2

In [ ]:
import pyspark.sql.functions as f

In [ ]:
df = (DFpeople
      .withColumn('Age', f.col('Age').cast(StringType())))

In [ ]:
df.printSchema()

###ładowanie df od razu ze zdefiniowana schema

In [ ]:
schema_for_people = StructType([StructField('Names', StringType()),
                                StructField('Age', IntegerType())])

In [ ]:
df_with_schema = spark.read.csv(path='people_noheader.txt', schema=schema_for_people)

In [ ]:
df_with_schema.printSchema()

### DF $\mapsto$ RDD

In [ ]:
df_with_schema.rdd.take(10)

In [ ]:
names = df_with_schema.rdd.map(lambda x: x.Names).collect()

In [ ]:
names

In [ ]:
names = df_with_schema.rdd.map(lambda x: x[0]).collect()

In [ ]:
names

In [ ]:
rdd_with_tuple = df_with_schema.rdd.map(lambda x: (x[0], x[1]))

In [ ]:
rdd_with_tuple.take(5)

In [ ]:
type(rdd_with_tuple)

##Zadanie 4
1. Załaduj plik pracownicy.csv z poprawnie zdefiniowaną strukturą i typami danych (schema)
2. Dla każdego roku urodzenia występującego w danych wylicz średnią stawkę godzinową
3. posortuj od najmłodszych lat do najstarszych
4. Wynik zapisz do excela

In [ ]:
#kod

#Zajęcia 25.04.2022

##UDF

In [ ]:
df = spark.createDataFrame([("2022-04-25", "John Doe", 21)], ("date", "name", "age"))
df.show()

In [ ]:
#Jak wyciąć rok z daty?

In [ ]:
from pyspark.sql.functions import udf

In [ ]:
def f_do_wycinania_roku(x):
  return x[:4]

In [ ]:
udf_do_wycinania_roku = udf(f_do_wycinania_roku, StringType())

In [ ]:
(df
 .withColumn('date', udf_do_wycinania_roku(f.col('date')))
 ).show()

In [ ]:
#Jeżeli chcemy UDF wykorzystywać później w SQL to trzeba go zarejestrować (3.3 Registering PySpark UDF & use it on SQL -> https://sparkbyexamples.com/pyspark/pyspark-udf-user-defined-function/)

In [ ]:
#""" Using UDF on SQL """
spark.udf.register("udf_do_wycinania_roku", udf_do_wycinania_roku)
df.createOrReplaceTempView("df")
spark.sql("select udf_do_wycinania_roku(date), name, age from df").show()

###Pagecounts dataset

In [ ]:
from pyspark.sql.types import *
import pyspark.sql.functions as f

In [ ]:
!unzip pagecounts.zip

In [ ]:
!head pagecounts/part-00001

In [ ]:
!head pagecounts/part-00053

In [ ]:
pagecounts = sc.textFile('pagecounts')

In [ ]:
pagecounts.count()

In [ ]:
pagecounts.take(10)

##Zadanie 5
Jakiego dnia ile było wejść a angielską wikipedię? Rozwiąż na dwa sposoby (wykorzystując czyste RDD oraz Dataframes). 4 kolumna w danych oznacza ile bylo wejsc.

In [ ]:
#kod 1 (RDD)

In [ ]:
#kod 2 (Dataframes)

##Zadanie 6
1. Wczytaj wiki_sample.parquet (moodle)
2. Odpowiedz na pytanie którzy użytkownicy edytowali najwięcej stron na wikipedii (top 5)?
3. Wykonaj zadanie na dwa sposoby (Dataframe SQL oraz Dataframe programmatic API)

In [ ]:
#kod

#Zajęcia 04.05.2022

##Funkcje agregujące

In [ ]:
df = spark.read.csv('sales_info.csv', inferSchema=True, header=True)

In [ ]:
df.show()

In [ ]:
df.printSchema()

In [ ]:
df.orderBy(['Company', 'Sales']).show()

In [ ]:
df.orderBy([f.col('Company'), f.col('Sales').desc()]).show()

In [ ]:
df.groupBy('Company').max().show()

In [ ]:
df.groupBy('Company').min().show()

In [ ]:
df.groupBy('Company').sum().show()

In [ ]:
(df
 .groupBy('Company')
 .agg(f.min('Sales').alias('MIN'),
      f.max('Sales').alias('MAX'),
      f.avg('Sales'),
      f.stddev('Sales'))
).withColumn('stddev_samp(Sales)', f.round(f.col('stddev_samp(Sales)'), 2)).show()

##Funkcje okienkowe

Sources: https://sparkbyexamples.com/pyspark/pyspark-window-functions/ , https://databricks.com/blog/2015/07/15/introducing-window-functions-in-spark-sql.html

Window functions explained: https://www.youtube.com/watch?v=wH6RaEWuiDQ (polecam!)

In [ ]:
import sys
from pyspark.sql.window import Window

In [ ]:
simpleData = (("James", "Sales", 3000), \
    ("Michael", "Sales", 4600),  \
    ("Robert", "Sales", 4100),   \
    ("Maria", "Finance", 3000),  \
    ("James", "Sales", 3000),    \
    ("Scott", "Finance", 3300),  \
    ("Jen", "Finance", 3900),    \
    ("Jeff", "Marketing", 3000), \
    ("Kumar", "Marketing", 2000),\
    ("Saif", "Sales", 4100) \
  )

In [ ]:
columns= ["employee_name", "department", "salary"]
df = spark.createDataFrame(data = simpleData, schema = columns)
df.printSchema()
df.show(truncate=False)
df.createOrReplaceTempView("df")

###row_number()

In [ ]:
from pyspark.sql.functions import row_number

In [ ]:
windowSpec = (Window
              .partitionBy(f.col('department'))
              .orderBy(f.col('salary').desc())
              )

In [ ]:
df.withColumn('row_number', row_number().over(windowSpec)).show()

In [ ]:
sqlDF = spark.sql("""SELECT employee_name, department, salary,
                     row_number() OVER(PARTITION BY department ORDER BY salary DESC) AS row_number
                     FROM df
""")

sqlDF.show()

###rank()

In [ ]:
from pyspark.sql.functions import rank

In [ ]:
df.withColumn("rank", rank().over(windowSpec)).show()

In [ ]:
sqlDF = spark.sql("""SELECT employee_name, department, salary,
                     rank() OVER(PARTITION BY department ORDER BY salary DESC) AS rank
                     FROM df
""")

sqlDF.show()

###dense_rank()

In [ ]:
from pyspark.sql.functions import dense_rank
df.withColumn("dense_rank", dense_rank().over(windowSpec)).show()

In [ ]:
sqlDF = spark.sql("""SELECT employee_name, department, salary,
                     dense_rank() OVER(PARTITION BY department ORDER BY salary DESC) AS dense_rank
                     FROM df
""")

sqlDF.show()

###min(), max()

In [ ]:
df.withColumn("min_departament", f.min('salary').over(windowSpec.rangeBetween(-sys.maxsize, sys.maxsize))).show()

In [ ]:
sqlDF = spark.sql(""" SELECT employee_name, department, salary,
                      min(salary) OVER(PARTITION BY department ORDER BY salary DESC RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as min_salary
                      FROM df
""")
sqlDF.show()

###LAG()

In [ ]:
import pyspark.sql.functions as f

In [ ]:
windowSpec = (Window
              .partitionBy(f.col('department'))
              .orderBy(f.col('salary').desc())
              )

In [ ]:
from pyspark.sql.functions import lag

In [ ]:
df.withColumn("lag_salary", lag("salary", 1).over(windowSpec)).show()

In [ ]:
sqlDF = spark.sql("""SELECT employee_name, department, salary,
                     LAG(salary) OVER(PARTITION BY department ORDER BY salary DESC) as lag_salary
                     FROM df

""")

In [ ]:
sqlDF.show(10)

###LEAD()

In [ ]:
from pyspark.sql.functions import lead

In [ ]:
df.withColumn("lead_salary", lead("salary", 2).over(windowSpec)).show()

In [ ]:
#SQL: https://www.youtube.com/watch?v=tNdJKrwQKIU

##Zadanie 7
1. wczytaj plik bonuses.csv
2. stwórz tabelę o kolumnach employee_id, year, max_quarter_bonus, min_quarter_bonus, total_bonus, total_bonus_previous_year, total_bonus_2_years_ago

In [ ]:
#kod

#Zajęcia 22.05.2024

##Machine Learning

##Colaborative Filtering
Dla chętnych: https://www.youtube.com/watch?v=ZspR5PZemcs

In [ ]:
!unzip ml-100k.zip

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base         
  inflating: ml-100k/u2.test         
  inflating: ml-100k/u3.base         
  inflating: ml-100k/u3.test         
  inflating: ml-100k/u4.base         
  inflating: ml-100k/u4.test         
  inflating: ml-100k/u5.base         
  inflating: ml-100k/u5.test         
  inflating: ml-100k/ua.base         
  inflating: ml-100k/ua.test         
  inflating: ml-100k/ub.base         
  inflating: ml-100k/ub.test         


In [ ]:
#u.data     -- The full u data set, 100000 ratings by 943 users on 1682 items.
#              Each user has rated at least 20 movies.  Users and items are
#              numbered consecutively from 1.  The data is randomly
#              ordered. This is a tab separated list of
#	         user id | item id | rating | timestamp.
#              The time stamps are unix seconds since 1/1/1970 UTC

In [ ]:
!head -5 ml-100k/u.data

196	242	3	881250949
186	302	3	891717742
22	377	1	878887116
244	51	2	880606923
166	346	1	886397596


In [ ]:
ratings_data = spark.read.csv("ml-100k/u.data", sep="	", header=False, inferSchema=True)

In [ ]:
from pyspark.sql.types import *

In [ ]:
fields = [StructField('user_id', IntegerType()),
          StructField('item_id', IntegerType()),
          StructField('rating', IntegerType()),
          StructField('timestamp', IntegerType())
          ]

In [ ]:
schema = StructType(fields)

In [ ]:
ratings_data = spark.read.csv("ml-100k/u.data", sep="	", header=False, schema=schema)

In [ ]:
ratings_data.show(10)

+-------+-------+------+---------+
|user_id|item_id|rating|timestamp|
+-------+-------+------+---------+
|    196|    242|     3|881250949|
|    186|    302|     3|891717742|
|     22|    377|     1|878887116|
|    244|     51|     2|880606923|
|    166|    346|     1|886397596|
|    298|    474|     4|884182806|
|    115|    265|     2|881171488|
|    253|    465|     5|891628467|
|    305|    451|     3|886324817|
|      6|     86|     3|883603013|
+-------+-------+------+---------+
only showing top 10 rows



In [ ]:
(train, test) = ratings_data.randomSplit([0.8, 0.2])

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

In [ ]:
#https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.recommendation.ALS.html#pyspark.ml.recommendation.ALS
als = ALS(maxIter=5, #ile razy powtórzy uczenie się
          modelu rank=20, regParam=0.01, userCol="user_id", itemCol="item_id", ratingCol="rating", coldStartStrategy="drop") #gdy klient nie obwejrzał żadnego filmu- dlatego wybieramy kilka filmów na start)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (<ipython-input-27-eb5b7c3e44b0>, line 3)

In [ ]:
model = als.fit(train)

NameError: name 'als' is not defined

In [ ]:
predictions = model.transform(test)

In [ ]:
predictions.show(10)

In [ ]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")

In [ ]:
rmse = evaluator.evaluate(predictions)

In [ ]:
print(f"Root-mean-square error = {rmse}")

In [ ]:
#top 10 movies for each user
userRecs = model.recommendForAllUsers(10)

In [ ]:
userRecs.show(10, False)

In [ ]:
#top 10 user for each movie
movieRecs = model.recommendForAllItems(10)

In [ ]:
movieRecs.show(10, False)

##Classification
###Decision Trees

In [ ]:
!unzip train.csv.zip

unzip:  cannot find or open train.csv.zip, train.csv.zip.zip or train.csv.zip.ZIP.


In [ ]:
data = spark.read.csv("train.csv", header=True, inferSchema=True, sep=",")

In [ ]:
data.show(10)

+---+---------+------+-----+--------------------------------+------------------------------+-------------------------------+-------------+--------------+-------------+----------------------------------+----------------+----------------+----------------+----------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+----------+
| Id|Elevation|Aspect|Slope|Horizontal_Distance_To_Hydrology|Vertical_Distance_To_Hydrology|Horizontal_Distance_To_Roadways|Hillshade_9am|Hillshade_Noon|Hillshade_3pm|Horizontal_Distance_To_Fire_Points|Wilderness_Area1|Wilderness_Area2|Wilderness_

In [ ]:
(train, test) = data.randomSplit([0.8, 0.2])

|y|x1|x2|x3
|-|-|-|-|
|5|1|2|3|
|3|5|6|7|

|y|X_i|
|-|---|
|5|[1,2,3]|
|3|[5,6,7]|

In [ ]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=[x for x in data.columns if x not in ['Id', 'Cover_Type']], outputCol='features')

In [ ]:
df_assemled_preview = assembler.transform(train)

In [ ]:
df_assemled_preview.show(10)

+---+---------+------+-----+--------------------------------+------------------------------+-------------------------------+-------------+--------------+-------------+----------------------------------+----------------+----------------+----------------+----------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+----------+--------------------+
| Id|Elevation|Aspect|Slope|Horizontal_Distance_To_Hydrology|Vertical_Distance_To_Hydrology|Horizontal_Distance_To_Roadways|Hillshade_9am|Hillshade_Noon|Hillshade_3pm|Horizontal_Distance_To_Fire_Points|Wilderness_Area1|Wildern

In [ ]:
df_assemled_preview.rdd.take(1)

[Row(Id=1, Elevation=2596, Aspect=51, Slope=3, Horizontal_Distance_To_Hydrology=258, Vertical_Distance_To_Hydrology=0, Horizontal_Distance_To_Roadways=510, Hillshade_9am=221, Hillshade_Noon=232, Hillshade_3pm=148, Horizontal_Distance_To_Fire_Points=6279, Wilderness_Area1=1, Wilderness_Area2=0, Wilderness_Area3=0, Wilderness_Area4=0, Soil_Type1=0, Soil_Type2=0, Soil_Type3=0, Soil_Type4=0, Soil_Type5=0, Soil_Type6=0, Soil_Type7=0, Soil_Type8=0, Soil_Type9=0, Soil_Type10=0, Soil_Type11=0, Soil_Type12=0, Soil_Type13=0, Soil_Type14=0, Soil_Type15=0, Soil_Type16=0, Soil_Type17=0, Soil_Type18=0, Soil_Type19=0, Soil_Type20=0, Soil_Type21=0, Soil_Type22=0, Soil_Type23=0, Soil_Type24=0, Soil_Type25=0, Soil_Type26=0, Soil_Type27=0, Soil_Type28=0, Soil_Type29=1, Soil_Type30=0, Soil_Type31=0, Soil_Type32=0, Soil_Type33=0, Soil_Type34=0, Soil_Type35=0, Soil_Type36=0, Soil_Type37=0, Soil_Type38=0, Soil_Type39=0, Soil_Type40=0, Cover_Type=5, features=SparseVector(54, {0: 2596.0, 1: 51.0, 2: 3.0, 3: 25

In [ ]:
from pyspark.ml.feature import StringIndexer
labelIndexer = StringIndexer(inputCol='Cover_Type', outputCol='label').fit(data)

In [ ]:
df_indexed_prview = labelIndexer.transform(train)
df_indexed_prview.show(10)

+---+---------+------+-----+--------------------------------+------------------------------+-------------------------------+-------------+--------------+-------------+----------------------------------+----------------+----------------+----------------+----------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+----------+-----+
| Id|Elevation|Aspect|Slope|Horizontal_Distance_To_Hydrology|Vertical_Distance_To_Hydrology|Horizontal_Distance_To_Roadways|Hillshade_9am|Hillshade_Noon|Hillshade_3pm|Horizontal_Distance_To_Fire_Points|Wilderness_Area1|Wilderness_Area2|Wilde

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier

In [ ]:
dt = DecisionTreeClassifier(impurity='gini',
                            maxDepth=5,
                            maxBins=32,
                            featuresCol='features',
                            labelCol='label')

In [ ]:
from pyspark.ml import Pipeline

In [ ]:
pipeline = Pipeline(stages=[assembler, labelIndexer, dt])

In [ ]:
model_pipeline = pipeline.fit(train)

In [ ]:
model_pipeline.stages

[VectorAssembler_04b187e21bd1,
 StringIndexerModel: uid=StringIndexer_8d7e8eb0102f, handleInvalid=error,
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_e6970620f641, depth=5, numNodes=55, numClasses=7, numFeatures=54]

In [ ]:
treeModel = model_pipeline.stages[2]

In [ ]:
treeModel

DecisionTreeClassificationModel: uid=DecisionTreeClassifier_e6970620f641, depth=5, numNodes=55, numClasses=7, numFeatures=54

In [ ]:
print(treeModel.toDebugString)

DecisionTreeClassificationModel: uid=DecisionTreeClassifier_e6970620f641, depth=5, numNodes=55, numClasses=7, numFeatures=54
  If (feature 0 <= 2678.5)
   If (feature 0 <= 2376.5)
    If (feature 6 <= 192.5)
     If (feature 3 <= 15.0)
      Predict: 3.0
     Else (feature 3 > 15.0)
      If (feature 30 <= 0.5)
       Predict: 2.0
      Else (feature 30 > 0.5)
       Predict: 3.0
    Else (feature 6 > 192.5)
     If (feature 3 <= 15.0)
      If (feature 12 <= 0.5)
       Predict: 3.0
      Else (feature 12 > 0.5)
       Predict: 5.0
     Else (feature 3 > 15.0)
      Predict: 3.0
   Else (feature 0 > 2376.5)
    If (feature 10 <= 0.5)
     If (feature 17 <= 0.5)
      If (feature 23 <= 0.5)
       Predict: 2.0
      Else (feature 23 > 0.5)
       Predict: 5.0
     Else (feature 17 > 0.5)
      If (feature 1 <= 86.5)
       Predict: 5.0
      Else (feature 1 > 86.5)
       Predict: 2.0
    Else (feature 10 > 0.5)
     If (feature 31 <= 0.5)
      If (feature 8 <= 80.5)
       Predict: 4

In [ ]:
predictions = model_pipeline.transform(test)

In [ ]:
predictions.show(10, False)

+---+---------+------+-----+--------------------------------+------------------------------+-------------------------------+-------------+--------------+-------------+----------------------------------+----------------+----------------+----------------+----------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+----------+------------------------------------------------------------------------------------------------------+-----+---------------------------------+--------------------------------------------------------------------------------------------+----------+


In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName='accuracy')

In [ ]:
accuracy = evaluator.evaluate(predictions)

In [ ]:
print(accuracy)

0.6497478991596639


##Random Forest

##Zadanie 8
Do tego samego zbioru danych stworz model klasyfikacji oparty na Random Forest (https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.ml.classification.RandomForestClassifier.html). Postaraj się zrobić go w formie "pipeline".

In [ ]:
#kod

#Zajęcia 16.05.2022
omowic labelConverter + hyperparameter tuning

##Hyperparameter Tuning

###Random Forest

In [ ]:
train.cache()
test.cache()

In [ ]:
train.count()
test.count()

In [ ]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [ ]:
paramGrid = (ParamGridBuilder()
  .addGrid(rf.numTrees, [10, 20])
  .addGrid(rf.maxDepth, [30])
  .addGrid(rf.maxBins, [100])
  .addGrid(rf.impurity, ['entropy'])
  .build()
)

In [ ]:
paramGrid

In [ ]:
pipeline = Pipeline(stages=[assembler, labelIndexer, rf, labelConverter])

In [ ]:
crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=3) # use 10 folds in practice

In [ ]:
# Run cross-validation, and choose the best set of parameters.
cvModel = crossval.fit(train)

In [ ]:
cvModel.bestModel

In [ ]:
params = [{p.name: v for p, v in m.items()} for m in cvModel.getEstimatorParamMaps()]
params

In [ ]:
cvModel.avgMetrics

In [ ]:
best_model = cvModel.bestModel

In [ ]:
predictions_test = best_model.transform(test)

In [ ]:
accuracy_test = evaluator.evaluate(predictions_test)

In [ ]:
print(f"Accuracy test: {accuracy_test}")

In [ ]:
predictions_train = best_model.transform(train)
accuracy_train = evaluator.evaluate(predictions_train)
print(f"Accuracy train: {accuracy_train}")

##Zadanie 9
Zaimplementuj hyperparameter tuning dla drzew decyzyjnych. Parametry:

impurity, ["entropy"];
maxDepth, [10, 30];
maxBins, [30, 300, 100];

In [ ]:
#kod

##Clustering
Based on: https://towardsdatascience.com/k-means-clustering-using-pyspark-on-big-data-6214beacdc8b
K-Means explained: https://www.youtube.com/watch?v=4b5d3muPQmA

In [ ]:
!unzip archive.zip

In [ ]:
data_customer=spark.read.csv('CC GENERAL.csv', header=True, inferSchema=True)
data_customer.printSchema()

In [ ]:
data_customer=data_customer.na.drop()

In [ ]:
from pyspark.ml.feature import VectorAssembler
data_customer.columns
assemble=VectorAssembler(inputCols=[
 'BALANCE',
 'BALANCE_FREQUENCY',
 'PURCHASES',
 'ONEOFF_PURCHASES',
 'INSTALLMENTS_PURCHASES',
 'CASH_ADVANCE',
 'PURCHASES_FREQUENCY',
 'ONEOFF_PURCHASES_FREQUENCY',
 'PURCHASES_INSTALLMENTS_FREQUENCY',
 'CASH_ADVANCE_FREQUENCY',
 'CASH_ADVANCE_TRX',
 'PURCHASES_TRX',
 'CREDIT_LIMIT',
 'PAYMENTS',
 'MINIMUM_PAYMENTS',
 'PRC_FULL_PAYMENT',
 'TENURE'], outputCol='features')
assembled_data=assemble.transform(data_customer)
assembled_data.show(2)

In [ ]:
from pyspark.ml.feature import StandardScaler
scale=StandardScaler(inputCol='features',outputCol='standardized')
data_scale=scale.fit(assembled_data)
data_scale_output=data_scale.transform(assembled_data)
data_scale_output.show(2)

In [ ]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
silhouette_score=[]
evaluator = ClusteringEvaluator(predictionCol='prediction', featuresCol='standardized', \
                                metricName='silhouette', distanceMeasure='squaredEuclidean')
for i in range(2,10):

    KMeans_algo=KMeans(featuresCol='standardized', k=i)

    KMeans_fit=KMeans_algo.fit(data_scale_output)

    output=KMeans_fit.transform(data_scale_output)



    score=evaluator.evaluate(output)

    silhouette_score.append(score)

    print("Silhouette Score:",score)

In [ ]:
#Visualizing the silhouette scores in a plot
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1,1, figsize =(8,6))
ax.plot(range(2,10),silhouette_score)
ax.set_xlabel('k')
ax.set_ylabel('cost')

In [ ]:
KMeans_algo=KMeans(featuresCol='standardized', k=3)
KMeans_fit=KMeans_algo.fit(data_scale_output)
output=KMeans_fit.transform(data_scale_output)
output.show(10)

In [ ]:
from pyspark.ml.feature import PCA
import matplotlib.pyplot as plt

pca = PCA(k=3, inputCol="features", outputCol="pcaFeatures")
model = pca.fit(data_scale_output)

result = model.transform(data_scale_output).select("pcaFeatures")
pandasDf = result.toPandas()
dataX = []
dataY = []

for vec in pandasDf.values:
  dataX.extend([vec[0][0]])
  dataY.extend([vec[0][1]])
plt.scatter(dataX, dataY)
plt.show()

#Zajęcia 23.05.2022

##Data Science How-To: Using Apache Spark for Sports Analytics
based on: https://tanzu.vmware.com/content/blog/how-data-science-assists-sports

In [ ]:
!unzip data.zip

In [ ]:
df = spark.read.option('header','true')\
          .option('inferSchema','true')\
          .csv('data/season_totals.csv')

In [ ]:
df.limit(5).show()

In [ ]:
import pyspark.sql.functions as f

In [ ]:
df.orderBy(f.col('pts').desc()).limit(10).show()

In [ ]:
#Ile było prób rzutu za 3 punkty na 36 minut

In [ ]:
fga_py = (df
          .groupBy('yr')
          .agg(f.sum('mp'), #minutes played (wszystkie rozegrane minuty wszystkich graczy)
               f.sum('fg3a')) #attempts
          .select(f.col('yr'), (36*f.col('sum(fg3a)')/f.col('sum(mp)')).alias('fg3a_p36m'))
          .orderBy('yr')
               )
fga_py.show(10)

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns

sns.set_style("white")
sns.set_color_codes()

plt.figure(figsize=(20,10))
plt.style.use('fivethirtyeight')

_df = fga_py.toPandas()
plt.plot(_df.yr,_df.fg3a_p36m, color = '#00a79c')
plt.xlabel('Year')
plt.title('Player average 3-point attempts (per 36 minutes)')
plt.annotate('3 pointer introduced', xy=(1980, .5), xytext=(1981, 1.1), fontsize = 12,
            arrowprops=dict(facecolor='grey', shrink=0.05, linewidth = 2))
plt.annotate('NBA moved in 3-point line', xy=(1996, 2.4), xytext=(1991.5, 2.7), fontsize = 12,
            arrowprops=dict(facecolor='grey', shrink=0.05, linewidth = 2))
plt.annotate('NBA moved back\n3-point line', xy=(1998, 2.), xytext=(1998.5, 2.4), fontsize = 12, arrowprops=dict(facecolor='grey', shrink=0.05, linewidth = 2))
plt.show()

###Zadanie 10
Regresja liniowa w Sparku - ile będzie rzutów na koniec 2022? Użyj Regresji Logistycznej

In [ ]:
#kod

###Heatmap 3-point shoots

In [ ]:
#statystyki skąd rzuty
df = spark.read.csv(path='data/shot_charts_top_10/1000_plus_shot_charts_2011_2016.csv', header=True, inferSchema=True)

In [ ]:
df.cache()
df.count()

In [ ]:
df.show()

In [ ]:
from nba_utils import draw_3pt_piechart,plot_shot_chart
import numpy as np
player = 'Stephen Curry'
yr = '2016'
df_steph = df.filter(f'''name == "{player}"
                        and yr == {yr}
                        and y < 400''')
#https://www.programiz.com/python-programming/list-comprehension

x = np.array([v[0] for v in df_steph.select('x').collect()])
y = np.array([v[0] for v in df_steph.select('y').collect()])
p=plot_shot_chart(x, y, gridsize = 30,
                      kind='hex')
# p.savefig('results/steph_curry_2016_shotchart.png')
plt.show()

In [ ]:
import warnings
warnings.simplefilter("ignore")

In [ ]:
df.show(5)

In [ ]:
def is_corner_3(xy):
    '''Want to identify corner 3 point attempts'''
    x,y = xy
    return int((abs(x) >= 220) and (y < 92.5))

def is_normal_3(xycorner3):
    '''Want to identify normal (not corner 3) point attempts'''
    x,y,corner3 = xycorner3
    radius = 475/2.
    y_3pt = np.sqrt(np.square(radius) - np.square(x))
    return int(y > max(92.5,y_3pt) and not corner3)

In [ ]:
from pyspark.sql.functions import array,udf
from pyspark.sql.types import IntegerType

corner_3_udf = udf(is_corner_3, IntegerType())
normal_3_udf = udf(is_normal_3, IntegerType())

df2 = df.withColumn('corner_3', corner_3_udf(array([df.x,df.y])))
df3 = df2.withColumn('normal_3', normal_3_udf(array([df2.x,df2.y,df2.corner_3])))
df4 = df3.withColumn('is_a_3', df3.corner_3 + df3.normal_3)

df = df4
df.cache()

In [ ]:
from pyspark.sql.functions import mean
from nba_utils import draw_3pt_piechart,plot_shot_chart

# make shot charts for all years
midrange_thresh = 8
in_half_court = 'y <= 400 and abs(x) <= 250'
addl_filter = f'shot_distance > {midrange_thresh}'
for yr in range(2011,2016+1):
    df_yr = df.filter(f'''{in_half_court}
                         and yr == {yr}
                         and {addl_filter}''')
    x = np.array([v[0] for v in df_yr.select('x').collect()])
    y = np.array([v[0] for v in df_yr.select('y').collect()])
    p = plot_shot_chart(x,y, gridsize = 30, kind = 'hex', label = yr)
    p.fig.suptitle('Evolution of the 3 point shot', x = .19, y = 0.86, size = 20, fontweight= 'bold')

    per_3 = df_yr.select(mean(df_yr.is_a_3)).take(1)[0][0]
    per_midrange = 1 - per_3
    draw_3pt_piechart(per_3, per_midrange)
    plt.show()
#     p.savefig('results/all_years/{}.png'.format(yr))
#     plt.close()

###Shots efficiency

In [ ]:
from pyspark.sql.functions import count, when

shot_acc = df.groupBy('shot_distance','corner_3','normal_3','is_a_3')\
             .agg(count('*').alias('num_attempts'),mean(df.shot_made_flag).alias('shot_accuracy'))\
             .withColumn('points_per_shot',when(f.col('is_a_3') == 1, f.col('shot_accuracy')*3)
                                           .otherwise(f.col('shot_accuracy')*2)
                        )\
             .filter('num_attempts > 5')\
             .orderBy('shot_distance')\
             .toPandas()

In [ ]:
df.show(3)

In [ ]:
shot_acc

In [ ]:
plt.style.use('fivethirtyeight')
plt.figure(figsize = (10,6))

def plot_acc_vs_dist(df,kwargs = {}):
    plt.plot(df.shot_distance, df.points_per_shot, **kwargs)

plot_acc_vs_dist(shot_acc.query('is_a_3 == False'), {'color' : '#008fd5'})
plot_acc_vs_dist(shot_acc.query('is_a_3 == True'), {'color' : '#008fd5'})
plt.title('Shot value vs. shot distance, 2011-2016 seasons\n Players with 1000+ attempts in a season')
plt.xlim(0,30)
plt.xlabel('Shot Distance (ft)')
plt.ylabel('Points per shot')
plt.annotate('high efficiency 2s', xy=(2., 1.15), xytext=(4.5, 1.28),
            arrowprops=dict(facecolor='grey', shrink=0.05))
plt.annotate('high efficiency 3s', xy=(22, 1.15), xytext=(14.5, 1.15),
            arrowprops=dict(facecolor='grey', shrink=0.05))
plt.text(22, 1.25, 'corner 3s', fontsize = 12)
#plt.savefig('results/pps.png')